In [ ]:
## ref : https://github.com/philschmid/deep-learning-pytorch-huggingface/blob/main/training/optimize-llama-2-gptq.ipynb 

In [ ]:
!sudo pip install -q transformers --upgrade

In [ ]:
!sudo -H pip install auto-gptq --no-cache-dir

In [ ]:
!sudo -H pip install --upgrade optimum

In [ ]:
import torch
torch.cuda.empty_cache()

In [ ]:
import gc
gc.collect()

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from optimum.gptq import GPTQQuantizer, load_quantized_model
import torch

In [ ]:
model_name = "mistralai/Mistral-7B-v0.1"

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [ ]:
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.float16,device_map='auto')

In [ ]:
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:512"

In [ ]:
quantizer = GPTQQuantizer(bits=4, dataset="wikitext2")
quantizer.quant_method = "gptq"

In [ ]:
quantized_model = quantizer.quantize_model(model, tokenizer)

In [ ]:
print(os.environ["PYTORCH_CUDA_ALLOC_CONF"])

In [ ]:
quant_path = "/data/quantization-trials/GPTQ-quantized/ravi"

In [ ]:
# save the quantize model to disk

quantized_model.save_pretrained(quant_path, safe_serialization=True)

### Inference on quantized model

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, GPTQConfig

In [ ]:
gptq_config = GPTQConfig(bits=4, use_exllama=True)

model_id = "/data/quantization-trials/GPTQ-quantized"
quant_model = AutoModelForCausalLM.from_pretrained(model_id, device_map="cuda", torch_dtype=torch.float16)

In [ ]:
def predict_from_quant(user_query):
    _inputs = tokenizer.encode(user_query, return_tensors="pt").to('cuda')
    outputs = quant_model.generate(input_ids=_inputs, max_length= 1000, pad_token_id=tokenizer.eos_token_id)
    output = tokenizer.decode(outputs[0])
    return output

import time

In [ ]:
# Using quant model
start = time.time()
output1 = predict_from_quant("what is science")
print("time taken is :", time.time()-start)

In [ ]:
def predict_from_normal(user_query):
    _inputs = tokenizer.encode(user_query, return_tensors="pt").to('cuda')
    outputs = model.generate(input_ids=_inputs, max_length= 1000, pad_token_id=tokenizer.eos_token_id)
    output = tokenizer.decode(outputs[0])
    return output

In [ ]:
# Using original model
start = time.time()
output1 = predict_from_normal("what is science")
print("time taken is :", time.time()-start)